In [1]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import os
import pickle
import time
from keras import backend as K
from sklearn.model_selection import cross_val_score
from sklearn.tree import ExtraTreeRegressor
from sklearn.metrics import r2_score
#from multiprocessing import Pool,cpu_count
import sklearn.metrics as m
from collections import Counter
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.metrics import mean_squared_error,mean_absolute_error
from sklearn.feature_selection import RFE

Using TensorFlow backend.


In [2]:
'''
Function that takes in a dataframe and then returns the dictionary of accuracies.
'''
'r'
req_columns = ["timestamp","air1","furnace1","waterheater1","total"]

def plot_test(y_test,y_pred,start,end):
    #Plot of testing.
    plt.figure(figsize=(10,10))
    plt.plot(y_test[start:end],'r')
    plt.plot(y_pred[start+1:end],'b')
    plt.xticks(np.arange(0,end-start,1))
    plt.legend()
    plt.show()

def coeff_determination(y_true, y_pred):
    SS_res =  K.sum(K.square( y_true-y_pred )) 
    SS_tot = K.sum(K.square( y_pred - K.mean(y_true) ) ) 
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

def create_df(localdf):
    i=0
    flag=0
    newdf = []
    timestamp = []
    d = np.zeros((7,))
    newhr = []
    
    while(24*(i+1)<localdf.shape[0]):
        #print (np.shape(np.sum(localdf.values[24*i:24*(i+1),1:],axis=0).reshape(7,1)),np.sum(localdf.values[24*i:24*(i+1),1:],axis=0).reshape(7,1))
        d = np.sum(localdf.values[24*i:24*(i+1),1:],axis=0)
        newhr.append(localdf.values[24*i:24*(i+1),-1])
        timestamp.append(localdf.values[24*i,0])
        newdf.append(d.T)
        i += 1
    
    print ("End i: ",i)
    
    newdf = np.hstack((np.reshape(timestamp,(-1,1)),newdf))
    newsum = []
    
    #If the dataframe pertaining to the house-id : 1632 is passed, then do just the following.
    if(localdf.shape[1]>=8):
        print ("Passed the original DataFrame corresponding to the house id : 1632.")
        flag=1
        d = pd.DataFrame(newdf,columns = localdf.columns)  
    
    #If the dataframe passed does not pertain to that of the house-id : 1632.
    if (flag!=1):
        d = pd.DataFrame(newdf,columns=["timestamp","air1","furnace1","waterheater1"])
        d['total_load'] = d["air1"] + d["furnace1"] + d["waterheater1"]
    
    print (d.head())
    newdf = d.values
    
    print (newdf.shape)
    return newdf,newhr
  


def extra_trees(localdf,localdict,times,metric,num_cols,hour_day,full=True,stop=3):
    for counter in range(times):
            i=num_cols
            df = pd.DataFrame()
            #The first column shouldn't be the 'timestamp'.
            if(i != 0):
                #Function returns Day loads and total load also separately
                ndf,nhr = create_df(localdf)
                print ("Inside extra trees : ",ndf.shape)
                if (full==True):
                    ndf = pd.DataFrame(ndf,columns = localdf.columns)
                else:
                    ndf = pd.DataFrame(ndf,columns = req_columns)
                
                #To get the total load of the days
                df = pd.DataFrame(ndf.values[:,-1])
                
                # To collate d1 to d7
                ndf = pd.concat([df.shift(l) for l in range(stop,-1,-1) ],axis=1)
                ndf.columns = [ "t-" + str(j) for j in range(stop,-1,-1) ]
                print ("mean :",ndf[ndf.columns[-1]].mean(),"max:",ndf[ndf.columns[-1]].max(),"min:",ndf[ndf.columns[-1]].min())
                ndf.rename(columns={'t-0':'t'}, inplace=True)
                
                #Adding new features.
                temp = pd.DataFrame()
                temp["dow"] = pd.to_datetime(ndf[ndf.columns[0]]).dt.dayofweek
                temp["month"] = pd.to_datetime(ndf[ndf.columns[0]]).dt.month
                    
                print ("Dimensions of temp : ",temp.shape)
                
                print ("\n\n Dimensions of ndf : ",ndf.shape)
                
                ndf = pd.concat([temp,ndf],axis=1)
                
                x = ndf.values[stop:,:-1]  #np.hstack((ndf.values[7:-1],nhr[7:-1]))
                y = ndf.values[stop:,-1]
                
                ndf = ndf.iloc[stop:]
                print ("Shape of ndf after merging : ",ndf.shape)
                print (ndf.head())
                
                #'''
                #Addition of a new feature.
                
                #Incorporating the total load values of the previous four weeks corresponding to the same day.
                
                temp = pd.DataFrame()
                temp["prev_4_week-days"] = ndf["t"]
                k=0
                i=27
                while i < (ndf.shape[0]):
                    temp["prev_4_week-days"].values[i] = ndf["t"].values[i-6] + ndf["t"].values[i-13] + ndf["t"].values[i-20] + ndf["t"].values[i-27]
                    i += 1
                print ("Temp : ",temp.head())
                
                print ("Shape of ndf : ",ndf.shape)
                print ("Shape of temp : ",temp.shape)
                
                print ("ndf : ",ndf.head())
                print ("temp : ",temp.head())
                
                ndf = pd.concat([temp,ndf],axis=1)
                x = ndf.values[27:,:-1] #np.hstack((ndf.values[7:-1],nhr[7:-1]))
                y = ndf.values[27:,-1]
                
                ndf = ndf.iloc[27:]
                print ("Shape of ndf after merging : ",ndf.shape)
                print (ndf.head())
                
                #print ("X : ",x)
                #'''
                
                #print ("\n The DataFrame to be passed to the model. \n",x.head())
                print (x.shape,y.shape)
                print (ndf.describe())
                train_size = int(x.shape[0]*0.7)
                X_train = x[:train_size]
                y_train = y[:train_size]
                
                X_test = x[train_size:]
                y_test = y[train_size:]
                print ("Split done ",X_train.shape,X_test.shape,y_train.shape,y_test.shape)
                
                model=ExtraTreesRegressor(n_estimators=500,max_features=None,verbose=1)
                model.fit(X_train,y_train)
                y_pred = model.predict(X_test)
                
                importances = model.feature_importances_
                std = np.std([tree.feature_importances_ for tree in model.estimators_],
                             axis=0)
                indices = np.argsort(importances)[::-1]

                # Print the feature ranking
                print("Feature ranking according to the ETR : ")

                for f in range(x.shape[1]):
                    print("%d. feature %d (%f)" % (f + 1, indices[f], importances[indices[f]]))

                '''
                Performing feature ranking.
                '''
                
                selector = RFE(model,step = 1)
                selector = selector.fit(x,y)
                print ("Ranking of the features : ",selector.ranking_,"\n\n\n")
                
                
                #The minimum value of y_pred should be '0', it can't go below that, that'd be illogical.
                y_pred[y_pred<0]=0
                
                r2 = r2_score(y_test,y_pred)
                mse = mean_squared_error(y_test,y_pred)
                mae = mean_absolute_error(y_test,y_pred)
                
                print ("R2 Score : ",r2)
                print ("RMSE : ",np.sqrt(mse))
                print ("MAE : ",mae)
                
                localdict["R2"] = r2
                localdict["RMSE"] = np.sqrt(mse)
                localdict["MAE"] = mae       
                
                #Visualize a plot of 'ytest' VS 'ypred'.
                plot_test(y_test,y_pred,0,50)
                
    return localdict,y_test,y_pred  

def nn_model(localdf,localdict,times,metric,num_cols,hour_day,full=True):
    for counter in range(times):
            i=num_cols
            df = pd.DataFrame()
            if(i != 0):
                #Function returns Day loads and total load also separately
                ndf,nhr = create_df(localdf)
                print ("Inside nn : ",ndf.shape)
                
                if (full==True):
                    ndf = pd.DataFrame(ndf,columns = localdf.columns)
                else:
                    ndf = pd.DataFrame(ndf,columns = req_columns)
                
                print (ndf.head())
                #To get the total load of the days
                df = pd.DataFrame(ndf.values[:,-1])
                # To collate d1 to d7
                ndf = pd.concat([df.shift(7),df.shift(6),df.shift(5),df.shift(4),df.shift(3),df.shift(2),df.shift(1),df.shift(0)],axis=1)
                ndf.columns = ('t-7','t-6','t-5','t-4','t-3','t-2','t-1','t')
                print ("mean :",ndf['t'].mean(),"max:",ndf['t'].max(),"min:",ndf['t'].min())
                
                x = ndf.values[7:,:-1] #np.hstack((ndf.values[7:-1],nhr[7:-1]))
                y = ndf.values[7:,-1]
                
                print ("Input Dimensions : ",x.shape,y.shape)
                print (ndf.describe())
                train_size = int(x.shape[0]*0.7)
                X_train = x[:train_size]
                y_train = y[:train_size]
                    
                X_test = x[train_size:]
                y_test = y[train_size:]
                
                print ("Split done ",X_train.shape,X_test.shape,y_train.shape,y_test.shape,)
                
                model = Sequential()
                model.add(Dense(7,input_dim=7,kernel_initializer='normal', activation='relu'))
                #Output Layer.
                model.add(Dense(1,kernel_initializer='normal'))
                #Compile model.
                model.compile(loss='mean_squared_error',optimizer='adam',metrics=[metric])
                #Fit the model.
                history = model.fit(X_train,y_train,epochs=750,verbose=1)
                #Final Evaluation of the model.
                #model.evaluate returns the 'loss' and the 'metrics' for the model.
                scores = model.evaluate(X_test,y_test,verbose=1,batch_size=X_test.shape[0])
                y_pred = model.predict(X_test,batch_size=X_test.shape[0])
                
                y_pred[y_pred<0]=0
                
                r2 = r2_score(y_test[:-1],y_pred[1:])
                mse = mean_squared_error(y_test[:-1],y_pred[1:])
                mae = mean_absolute_error(y_test[:-1],y_pred[1:])
                
                print ("R2 Score : ",r2)
                print ("RMSE : ",np.sqrt(mse))
                print ("MAE : ",mae)
                
                localdict["R2"] = r2
                localdict["RMSE"] = np.sqrt(mse)
                localdict["MAE"] = mae       
                
                #Visualize a plot of 'ytest' VS 'ypred'.
                plot_test(y_test,y_pred,0,50)
                del model
                
    return localdict,y_test,y_pred  

In [3]:
df = pd.read_pickle("../DataSets/New_Pecan_Street/load_data_115.pkl")
df

,dataid,localhour,use,air1,air2,air3,airwindowunit1,aquarium1,bathroom1,bathroom2,...,refrigerator1,refrigerator2,security1,shed1,sprinkler1,utilityroom1,venthood1,waterheater1,waterheater2,winecooler1
0,115,2014-01-24 19:00:00-06,1.022517,0.00000,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN
1,115,2014-01-24 20:00:00-06,1.771083,0.00000,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN
2,115,2014-01-24 21:00:00-06,0.893550,0.00000,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN
3,115,2014-01-24 22:00:00-06,0.194233,0.00000,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN
4,115,2014-01-24 23:00:00-06,0.403667,0.00000,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,0.222317,NaN,NaN
5,115,2014-01-25 00:00:00-06,0.276450,0.00000,NaN,NaN,NaN,NaN,NaN,NaN,...,0.112483,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN
6,115,2014-01-25 01:00:00-06,0.155283,0.00000,NaN,NaN,NaN,NaN,NaN,NaN,...,0.006967,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN
7,115,2014-01-25 02:00:00-06,1.313833,0.00000,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN
8,115,2014-01-25 03:00:00-06,0.564883,0.00000,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN
9,115,2014-01-25 04:00:00-06,1.012517,0.00000,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,0.225217,NaN,NaN


In [4]:
pos=[]
for i in range(3,len(df.columns)):
    if (np.sum(np.isnan(df.iloc[:,i].values))>=(df.shape[0]/2)):
        pos.append(i)
print (df.columns[pos])

df.drop(df.columns[pos], inplace=True, axis=1)
for i in range(3,len(df.columns)):
    print (df.columns[i], np.sum(np.isnan(df.iloc[:,i].values)))    

Index(['air2', 'air3', 'airwindowunit1', 'aquarium1', 'bathroom1', 'bathroom2',
       'bedroom1', 'bedroom2', 'bedroom3', 'bedroom4', 'bedroom5', 'car1',
       'clotheswasher_dryg1', 'diningroom1', 'diningroom2', 'disposal1',
       'dryg1', 'freezer1', 'furnace2', 'garage1', 'garage2', 'gen', 'heater1',
       'housefan1', 'icemaker1', 'jacuzzi1', 'kitchen1', 'kitchen2',
       'kitchenapp2', 'lights_plugs1', 'lights_plugs2', 'lights_plugs3',
       'lights_plugs4', 'lights_plugs5', 'lights_plugs6', 'livingroom1',
       'livingroom2', 'office1', 'outsidelights_plugs1',
       'outsidelights_plugs2', 'oven1', 'oven2', 'pool1', 'pool2',
       'poollight1', 'poolpump1', 'pump1', 'refrigerator2', 'security1',
       'shed1', 'sprinkler1', 'utilityroom1', 'venthood1', 'waterheater2',
       'winecooler1'],
      dtype='object')
air1 0
clotheswasher1 0
dishwasher1 0
drye1 0
furnace1 0
grid 0
kitchenapp1 0
microwave1 0
range1 0
refrigerator1 0
waterheater1 0


In [5]:

df_thermal=df[['localhour','air1','furnace1','waterheater1']]
df_thermal['total_thermal_load'] = df['air1'] + df['furnace1'] + df['waterheater1'] 
df_thermal=pd.DataFrame(df_thermal)

/opt/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [6]:
df_thermal=df_thermal.set_index(df_thermal['localhour'])
df_thermal.index=pd.to_datetime(df_thermal.index)
df_thermal.drop('localhour',inplace=True, axis=1)
df_thermal *= 1000
print (type(df_thermal.index))
df_thermal

<class 'pandas.core.indexes.datetimes.DatetimeIndex'>


,air1,furnace1,waterheater1,total_thermal_load
localhour,,,,
2014-01-25 01:00:00,0.00,805.133333,0.000000,805.133333
2014-01-25 02:00:00,0.00,1572.700000,0.000000,1572.700000
2014-01-25 03:00:00,0.00,631.100000,0.000000,631.100000
2014-01-25 04:00:00,0.00,0.000000,0.000000,0.000000
2014-01-25 05:00:00,0.00,0.000000,222.316667,222.316667
2014-01-25 06:00:00,0.00,0.000000,0.000000,0.000000
2014-01-25 07:00:00,0.00,0.000000,0.000000,0.000000
2014-01-25 08:00:00,0.00,1162.933333,0.000000,1162.933333
2014-01-25 09:00:00,0.00,410.600000,0.000000,410.600000


In [7]:
df_new_thermal=df_thermal.resample('D').sum()
df_new_thermal

,air1,furnace1,waterheater1,total_thermal_load
localhour,,,,
2014-01-25,0.000000,19820.916667,1102.500000,20923.416667
2014-01-26,0.000000,5199.866667,1040.250000,6240.116667
2014-01-27,0.000000,18717.083333,1429.133333,20146.216667
2014-01-28,0.000000,38492.133333,1931.000000,40423.133333
2014-01-29,0.000000,32664.416667,1485.500000,34149.916667
2014-01-30,0.000000,16977.200000,1418.216667,18395.416667
2014-01-31,0.000000,1329.833333,1403.850000,2733.683333
2014-02-01,0.000000,0.000000,2096.650000,2096.650000
2014-02-02,0.000000,6847.216667,1186.833333,8034.050000


In [16]:
air_null=[]
fur_null=[]
wh_null=[]
for i in range(df_new_thermal.shape[0]):
    if df_new_thermal['air1'][i]==0:
        air_null.append(i)
    if df_new_thermal['furnace1'][i]==0:
        fur_null.append(i)
    if df_new_thermal['waterheater1'][i]==0:
        wh_null.append(i)
print(len(air_null),len(fur_null),len(wh_null))

440 251 14


In [19]:
df_new_thermal['furnace1'][fur_null]

localhour
2014-02-01    0.0
2014-02-15    0.0
2014-02-19    0.0
2014-02-21    0.0
2014-02-22    0.0
2014-02-23    0.0
2014-02-24    0.0
2014-02-25    0.0
2014-03-01    0.0
2014-03-02    0.0
2014-03-08    0.0
2014-03-11    0.0
2014-03-15    0.0
2014-03-16    0.0
2014-03-19    0.0
2014-03-20    0.0
2014-03-21    0.0
2014-03-22    0.0
2014-03-23    0.0
2014-03-26    0.0
2014-03-27    0.0
2014-03-29    0.0
2014-03-30    0.0
2014-04-05    0.0
2014-04-06    0.0
2014-04-07    0.0
2014-04-08    0.0
2014-04-09    0.0
2014-04-10    0.0
2014-04-11    0.0
             ... 
2016-04-06    0.0
2016-04-08    0.0
2016-04-09    0.0
2016-04-10    0.0
2016-04-14    0.0
2016-04-15    0.0
2016-04-16    0.0
2016-05-03    0.0
2016-05-04    0.0
2016-08-16    0.0
2016-10-01    0.0
2017-10-11    0.0
2017-10-17    0.0
2017-10-18    0.0
2017-10-19    0.0
2017-10-23    0.0
2017-10-25    0.0
2017-10-26    0.0
2017-10-27    0.0
2017-10-30    0.0
2017-10-31    0.0
2017-11-01    0.0
2017-11-02    0.0
2017-11-08    0.0
